In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

## Download price data from YahooFinance(1996-2022, S&P 500 Components)

In [3]:
import pandas as pd
import os
sp500_historical_component = pd.read_csv("S&P 500 Historical Components & Changes(03-14-2022).csv")

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
dates = list(sp500_historical_component["date"])
tickers = sp500_historical_component["tickers"]

dates.append("2022-03-03")
for i in range(len(dates) - 1):
    try:
      df = (YahooDownloader(start_date = str(dates[i]), end_date = str(dates[i+1]), ticker_list = tickers[i].split(",")).fetch_data())
      df.to_hdf("data.hdf", 'a')
    except:
      pass

In [54]:
price_df = pd.read_hdf("data.hdf")

## Preprocess fundamental data

In [4]:
fund_df = pd.read_csv("sp500_fundamental_data_1996_2022.csv")

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10,17,26,30,647,648,650,654,657,668,669) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
items = [
    'datadate', # Date
    'tic', # Ticker
    'oiadpq', # Quarterly operating income
    'revtq', # Quartely revenue
    'niq', # Quartely net income
    'atq', # Total asset
    'teqq', # Shareholder's equity
    'epspiy', # EPS(Basic) incl. Extraordinary items
    'ceqq', # Common Equity
    'cshoq', # Common Shares Outstanding
    'dvpspq', # Dividends per share
    'actq', # Current assets
    'lctq', # Current liabilities
    'cheq', # Cash & Equivalent
    'rectq', # Recievalbles
    'cogsq', # Cost of  Goods Sold
    'invtq', # Inventories
    'apq',# Account payable
    'dlttq', # Long term debt
    'dlcq', # Debt in current liabilites
    'ltq' # Liabilities   
]

# Omit items that will not be used
fund_data = fund_df[items]


In [6]:
# Rename column names for the sake of readability
fund_data = fund_data.rename(columns={
    'datadate':'date', # Date
    'oiadpq':'op_inc_q', # Quarterly operating income
    'revtq':'rev_q', # Quartely revenue
    'niq':'net_inc_q', # Quartely net income
    'atq':'tot_assets', # Assets
    'teqq':'sh_equity', # Shareholder's equity
    'epspiy':'eps_incl_ex', # EPS(Basic) incl. Extraordinary items
    'ceqq':'com_eq', # Common Equity
    'cshoq':'sh_outstanding', # Common Shares Outstanding
    'dvpspq':'div_per_sh', # Dividends per share
    'actq':'cur_assets', # Current assets
    'lctq':'cur_liabilities', # Current liabilities
    'cheq':'cash_eq', # Cash & Equivalent
    'rectq':'receivables', # Receivalbles
    'cogsq':'cogs_q', # Cost of  Goods Sold
    'invtq':'inventories', # Inventories
    'apq': 'payables',# Account payable
    'dlttq':'long_debt', # Long term debt
    'dlcq':'short_debt', # Debt in current liabilites
    'ltq':'tot_liabilities' # Liabilities   
})

In [7]:
# Calculate financial ratios
date = pd.to_datetime(fund_data['date'],format='%Y%m%d')

tic = fund_data['tic'].to_frame('tic')

# Profitability ratios
# Operating Margin
OPM = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='OPM')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        OPM[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        OPM.iloc[i] = np.nan
    else:
        OPM.iloc[i] = np.sum(fund_data['op_inc_q'].iloc[i-3:i])/np.sum(fund_data['rev_q'].iloc[i-3:i])

# Net Profit Margin        
NPM = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='NPM')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        NPM[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        NPM.iloc[i] = np.nan
    else:
        NPM.iloc[i] = np.sum(fund_data['net_inc_q'].iloc[i-3:i])/np.sum(fund_data['rev_q'].iloc[i-3:i])

# Return On Assets
ROA = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='ROA')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        ROA[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        ROA.iloc[i] = np.nan
    else:
        ROA.iloc[i] = np.sum(fund_data['net_inc_q'].iloc[i-3:i])/fund_data['tot_assets'].iloc[i]

# Return on Equity
ROE = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='ROE')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        ROE[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        ROE.iloc[i] = np.nan
    else:
        ROE.iloc[i] = np.sum(fund_data['net_inc_q'].iloc[i-3:i])/fund_data['sh_equity'].iloc[i]        

# For calculating valuation ratios in the next subpart, calculate per share items in advance
# Earnings Per Share       
EPS = fund_data['eps_incl_ex'].to_frame('EPS')

# Book Per Share
BPS = (fund_data['com_eq']/fund_data['sh_outstanding']).to_frame('BPS') # Need to check units

#Dividend Per Share
DPS = fund_data['div_per_sh'].to_frame('DPS')

# Liquidity ratios
# Current ratio
cur_ratio = (fund_data['cur_assets']/fund_data['cur_liabilities']).to_frame('cur_ratio')

# Quick ratio
quick_ratio = ((fund_data['cash_eq'] + fund_data['receivables'] )/fund_data['cur_liabilities']).to_frame('quick_ratio')

# Cash ratio
cash_ratio = (fund_data['cash_eq']/fund_data['cur_liabilities']).to_frame('cash_ratio')


# Efficiency ratios
# Inventory turnover ratio
inv_turnover = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='inv_turnover')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        inv_turnover[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        inv_turnover.iloc[i] = np.nan
    else:
        inv_turnover.iloc[i] = np.sum(fund_data['cogs_q'].iloc[i-3:i])/fund_data['inventories'].iloc[i]

# Receivables turnover ratio       
acc_rec_turnover = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='acc_rec_turnover')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        acc_rec_turnover[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        acc_rec_turnover.iloc[i] = np.nan
    else:
        acc_rec_turnover.iloc[i] = np.sum(fund_data['rev_q'].iloc[i-3:i])/fund_data['receivables'].iloc[i]

# Payable turnover ratio
acc_pay_turnover = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='acc_pay_turnover')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        acc_pay_turnover[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        acc_pay_turnover.iloc[i] = np.nan
    else:
        acc_pay_turnover.iloc[i] = np.sum(fund_data['cogs_q'].iloc[i-3:i])/fund_data['payables'].iloc[i]
        
## Leverage financial ratios
# Debt ratio
debt_ratio = (fund_data['tot_liabilities']/fund_data['tot_assets']).to_frame('debt_ratio')

# Debt to Equity ratio
debt_to_equity = (fund_data['tot_liabilities']/fund_data['sh_equity']).to_frame('debt_to_equity')

<ipython-input-7-217f6a06b836>:15: RuntimeWarning: divide by zero encountered in double_scalars
  OPM.iloc[i] = np.sum(fund_data['op_inc_q'].iloc[i-3:i])/np.sum(fund_data['rev_q'].iloc[i-3:i])
<ipython-input-7-217f6a06b836>:15: RuntimeWarning: invalid value encountered in double_scalars
  OPM.iloc[i] = np.sum(fund_data['op_inc_q'].iloc[i-3:i])/np.sum(fund_data['rev_q'].iloc[i-3:i])
<ipython-input-7-217f6a06b836>:25: RuntimeWarning: divide by zero encountered in double_scalars
  NPM.iloc[i] = np.sum(fund_data['net_inc_q'].iloc[i-3:i])/np.sum(fund_data['rev_q'].iloc[i-3:i])
<ipython-input-7-217f6a06b836>:25: RuntimeWarning: invalid value encountered in double_scalars
  NPM.iloc[i] = np.sum(fund_data['net_inc_q'].iloc[i-3:i])/np.sum(fund_data['rev_q'].iloc[i-3:i])
<ipython-input-7-217f6a06b836>:35: RuntimeWarning: divide by zero encountered in double_scalars
  ROA.iloc[i] = np.sum(fund_data['net_inc_q'].iloc[i-3:i])/fund_data['tot_assets'].iloc[i]
<ipython-input-7-217f6a06b836>:45: Runtim

In [8]:
# Create a dataframe that merges all the ratios
ratios = pd.concat([date,tic,OPM,NPM,ROA,ROE,EPS,BPS,DPS,
                    cur_ratio,quick_ratio,cash_ratio,inv_turnover,acc_rec_turnover,acc_pay_turnover,
                   debt_ratio,debt_to_equity], axis=1)

In [9]:
# Replace NAs infinite values with zero
final_ratios = ratios.copy()
final_ratios = final_ratios.fillna(0)
final_ratios = final_ratios.replace(np.inf,0)

## Merge stock price data and ratios into one dataframe


In [11]:
price_df["date"] = price_df["date"].astype(np.datetime64)

# Merge stock price data and ratios into one dataframe
processed_full = price_df
processed_full = processed_full.merge(final_ratios,how='left',on=['date','tic'])
processed_full = processed_full.sort_values(['tic','date'])

# Backfill the ratio data to make them daily
processed_full = processed_full.bfill(axis='rows')

In [12]:

# Calculate P/E, P/B and dividend yield using daily closing price
processed_full['PE'] = processed_full['close']/processed_full['EPS']
processed_full['PB'] = processed_full['close']/processed_full['BPS']
processed_full['Div_yield'] = processed_full['DPS']/processed_full['close']

# Drop per share items used for the above calculation
processed_full = processed_full.drop(columns=['day','EPS','BPS','DPS'])
# Replace NAs infinite values with zero
processed_full = processed_full.copy()
processed_full = processed_full.fillna(0)
processed_full = processed_full.replace(np.inf,0)

In [13]:
# Save processed data
processed_full.to_hdf("processed_full.hdf", key="full", complevel=9)

## Generate Quarterly Data According to Price and Fundamental Data

In [56]:
l_df = price_df.groupby("tic")

quarterlies = []
for tic, df in l_df:
    processed_quarterly = final_ratios[final_ratios["tic"] == tic]
    processed_quarterly.sort_values(['date'])
    processed_quarterly.reset_index(inplace=True, drop=True)
    processed_quarterly["open"] = np.nan
    processed_quarterly["high"] = np.nan
    processed_quarterly["close"] = np.nan  
    processed_quarterly["low"] = np.nan
    processed_quarterly["volume"] = np.nan
    df.sort_values(['date'])
    df["date"] = df["date"].astype(np.datetime64)
    df_date = df.date.to_numpy()
    processed_quarterly_date =  processed_quarterly.date.to_numpy()
    np_open = df["open"].to_numpy()
    np_close = df["close"].to_numpy()
    np_high = df["high"].to_numpy()
    np_low = df["low"].to_numpy()
    np_volume = df["volume"].to_numpy()
    pre = 0
    j = 0
    for i in range(len(processed_quarterly)):
        while (j < len(df)) and (df_date[j] <= processed_quarterly_date[i]):
            j += 1
        if j <= pre:
            continue
        processed_quarterly.open[i]= np_open[pre]
        processed_quarterly.close[i]= np_close[j-1]
        processed_quarterly.high[i] = max(np_high[pre:j])
        processed_quarterly.low[i] = min(np_low[pre:j])
        processed_quarterly.volume[i] = sum(np_volume[pre:j])
        pre = j
    processed_quarterly.dropna(inplace=True)
    quarterlies.append(processed_quarterly)

<ipython-input-56-44949c6d15d8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_quarterly["open"] = np.nan
<ipython-input-56-44949c6d15d8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_quarterly["high"] = np.nan
<ipython-input-56-44949c6d15d8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [57]:
processed_quarterly_full = pd.concat(quarterlies)

In [59]:
processed_quarterly_full.reset_index(drop=True, inplace=True)

In [62]:

# Calculate P/E, P/B and dividend yield using daily closing price
processed_quarterly_full['PE'] = processed_quarterly_full['close']/processed_quarterly_full['EPS']
processed_quarterly_full['PB'] = processed_quarterly_full['close']/processed_quarterly_full['BPS']
processed_quarterly_full['Div_yield'] = processed_quarterly_full['DPS']/processed_quarterly_full['close']

# Drop per share items used for the above calculation
processed_quarterly_full = processed_quarterly_full.drop(columns=['EPS','BPS','DPS'])
# Replace NAs infinite values with zero
processed_quarterly_full = processed_quarterly_full.copy()
processed_quarterly_full = processed_quarterly_full.fillna(0)
processed_quarterly_full = processed_quarterly_full.replace(np.inf,0)

In [63]:
processed_quarterly_full.to_csv("processed_quarterly_full.csv")